In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pandas as pd

# 1.

df = pd.read_csv('bike_daily.csv')

temp_c = df["temp_c"].values
humidity = df["humidity"].values
wind_kph = df["wind_kph"].values
is_holiday = df["is_holiday"].values

# interested in the int values of the seasons.
season = pm.Categorical(df["season"].values).codes

# extract the observed y.
rentals = df["rentals"].values

# scatter between priors and rentals.

plt.plot(temp_c, rentals, label="temperature affecting rentals")
plt.plot(humidity, rentals, label="humidity affecting rentals")
plt.plot(wind_kph, rentals, label="wind_kph affecting rentals")
plt.plot(is_holiday, rentals, label="is_holiday affecting rentals")
plt.plot(season, rentals, label="season affecting rentals")

# Non-linearities

These are initial expectations.

Humidity: Would not affect the rents at all

Temperature: For extreme conditions it would affect the overall rent, but small changes between 20 and 30 won't make a difference
Wind speed: For extreme conditions it would affect the overall rent, but small changes won't make a difference

Holiday: if its holiday people might have more free time and rent a bike
Season: in the winter we intially expect to be few rentals

In [ ]:
# 2.
# a)

temp_c_mean = temp_c.mean()
temp_c_sd = temp_c.std()
temp_c_standard = (temp_c - temp_c_mean) / temp_c_sd

humidity_mean = humidity.mean()
humidity_sd = humidity.std()
humidity_standard = (humidity - humidity_mean) / humidity_sd

wind_kph_mean = wind_kph.mean()
wind_kph_sd = wind_kph.std()
wind_kph_standard = (wind_kph - wind_kph_mean) / wind_kph_sd

# standardize also the observed.
rentals_mean = rentals.mean()
rentals_sd = rentals.std()
rentals_standard = (rentals - rentals_mean) / rentals_sd

# b)

# assuming that rentals follows a normal distribution
priors = [temp_c_standard, humidity_standard, wind_kph_standard, is_holiday, season]

with pm.Model() as model_default:
    sigma = pm.HalfNormal('sigma', sigma=5)
    
    alpha = pm.Normal('alpha', sigma=100)

    # one such beta for each prior.
    beta = pm.Normal('beta', mu=0, sigma=100, shape=5)
    
    mu = alpha
    for i, b in enumerate(beta):
        mu += b * priors[i]
    
    infered_rentals = pm.Normal('infered_rentals', mu=mu, sigma=sigma, observed=rentals_standard)

# c)

with pm.Model() as model_squared_temp:
    sigma = pm.HalfNormal('sigma', sigma=5)
    
    alpha = pm.Normal('alpha', sigma=1000)

    # greater shape since we add one more element.
    beta = pm.Normal('beta', mu=0, sigma=100, shape=6)
    
    mu = alpha
    for i, b in enumerate(beta):
        mu += b * priors[i]
    mu += beta[5] * temp_c_standard ** 2
    
    infered_rentals = pm.Normal('infered_rentals', mu=mu, sigma=sigma, observed=rentals_standard)

In [ ]:
# 3.

with model_default:
    trace_default = pm.sample(2000, return_inferencedata=True, chains=2, target_accept=0.9)
    az.summary(trace_default)

with model_squared_temp:
    trace_squared_temp = pm.sample(2000, return_inferencedata=True, chains=2, target_accept=0.9)
    az.summary(trace_squared_temp)

In [ ]:
# 4.
# a) compare models

# choose waic
# - more efficient
# - penalizes the high amount of parameters

waic_default = az.waic(trace_default)
print(waic_default)

waic_squared_temp = az.waic(trace_squared_temp)
print(waic_squared_temp)

# b) vizualize predicted mean and uncertainty

pm.sample_posterior_predictive(trace_default, model=model_default, random_seed=2, extend_inferencedata=True)
az.plot_ppc(trace_default, num_pp_samples=200, figsize=(12, 6), mean=True)

pm.sample_posterior_predictive(trace_squared_temp, model=model_squared_temp, random_seed=2, extend_inferencedata=True)
az.plot_ppc(trace_squared_temp, num_pp_samples=200, figsize=(12, 6), mean=True)

In [ ]:

# 5.

rentals_Q_data = np.percentile(rentals_standard, 75.0)

# 6.

# no temp_c ** 2 puts to much weigth on the posterior of the rentals.

with pm.Model() as model_binary:
    # infer again on the rentals.
    sigma = pm.HalfNormal('sigma', sigma=5)
    alpha = pm.Normal('alpha', sigma=100)
    beta = pm.Normal('beta', mu=0, sigma=100, shape=5)
    mu = alpha
    for i, b in enumerate(beta):
        mu += b * priors[i]
    infered_rentals = pm.Normal('infered_rentals', mu=mu, sigma=sigma, observed=rentals_standard)

    # benoulli for the is_high_demand.
    binary_alpha = pm.Normal('binary_alpha', mu=0, sigma=10)
    binary_beta = pm.Normal('binary_beta', mu=0, sigma=10)
    
    theta = pm.Deterministic('theta', infered_rentals.mean())
    bd = pm.Deterministic('bd', -binary_alpha/binary_alpha)

    # daca mai mare de 75 % percentile
    is_high_demand = pm.Bernoulli('is_high_demand', p=theta)
    trace_binary = pm.sample(2000, return_inferencedata=True)

# 7. 95% HDIs for coefficients

hdi = az.hdi(trace_binary, var_names=["binary_alpha", "binary_beta"], hdi_prob=0.95)
print("Coefficient HDIs (94%):")
print(hdi)